<a href="https://colab.research.google.com/github/nicolejlaurin/AStarSearch/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#COMP4106 Assignment 1

class node:
    def __init__(self, coords, value): #constructor to initilize values for node objects
        self.coords = coords
        self.value = value
        self.pathcost = 0
        self.totalcost = 0
        self.adjacent = []
        self.parent = None
        self.isExplored = False
    def __repr__(self):
        return '(' + ','.join(map(str,self.coords)) + ') : ' + str(list(map(str, self.adjacent)))
    def __str__(self):
        return '(' + ','.join(map(str,self.coords)) + ')' 
    def addAdjacent(self, node):      #adds nodes to adjacency list if it is not a wall
        if node.value != 'X' and node.value != 'x':
            self.adjacent.append(node)

def pathfinding(CSV, optimal, explored): # calls other functions and writes to the files
  #initialize empty lists to populate
  nodeArray = []
  goalArray = []
  exploredList = []
  frontier = []

  (startNode, smallestStep) = format_grid(CSV, nodeArray, goalArray)  #to format grid from input and get start node
  goalNode = traverse_grid(startNode, smallestStep, goalArray, frontier, exploredList) #traverse through grid to get goal node

  with open(explored, "w") as exploredFile:   #writes explored nodes to file and formats it
    for exp in exploredList:
      exploredFile.write(str(exp) + '\n')
    exploredFile.close()
  
  if goalNode == None:
    print("No path found from start to goal\n")
    with open(optimal, "w") as optimalFile:   #write error message to file
      optimalFile.write("No path found from start to goal")
      optimalFile.close() 
    return None
  else:
    optimalList = [goalNode]
    currNode = goalNode
    optimalCost = 0

    while currNode.parent != None:    #determines optimal path from goal to start
      if currNode.value != 'g' and currNode.value != 'G':
        optimalCost += currNode.value
      optimalList.append(currNode.parent)
      currNode = currNode.parent
    optimalList.reverse()             #reverse to format nodes from start to goal

    with open(optimal, "w") as optimalFile:   #writes optimal path nodes to file and formats it
      for opt in optimalList:
        optimalFile.write(str(opt) + '\n')
      optimalFile.close() 
    return optimalCost          #returns cost of the optimal path



def traverse_grid(node, smallestStep, goalList, frontier, explored): #function to traverse to other nodes.


  explored.append(node)
  node.isExplored = True

  if node.value == 'g' or node.value == 'G':   #checks if current node is the Goal node
    return node
  else:     # if its not a goal node, it iterates through grid
    for adj in node.adjacent:
      if adj.pathcost > -1 and not adj.isExplored and pathcost(adj, node) < adj.pathcost: #if node is in frontier and total path cost is less than path cost for adjacent node
        frontier.remove(adj)
        #Need to also update entire subtree's pathcosts
      if adj.pathcost > -1 and adj.isExplored and pathcost(adj, node) < adj.pathcost: #if node is in explored and total path cost is less than path cost for adjacent node
        adj.isExplored = False
        explored.remove(adj)

      if not adj.isExplored and not adj in frontier: #checks if node is neither in explored or frontier
        adj.pathcost = pathcost(adj, node)
        adj.parent = node
        adj.totalcost = node_cost(adj, node, smallestStep, goalList)
        frontier.append(adj)

    frontier = sorted(frontier, key= lambda node : node.totalcost)
    #If there are still nodes to explore, move to the next one
    if len(frontier) > 0:
      return traverse_grid(frontier.pop(0), smallestStep, goalList, frontier, explored)
    #If there are no further nodes to explore, there is no path to the goal.
    else:
      return None

def node_cost(node, parent, smallestStep, goalList): #outputs total cost to traverse to node

  path_cost = pathcost(node,parent)           #gets path cost from node to its parent
  heuristic_cost = scaledManhattanMinusOne(node, smallestStep, goalList)  #calculates the heuristic cost
  return path_cost + heuristic_cost 

def pathcost(node, parent): #outputs the path cost from parent node to next node
  step_cost = 0
  if node.value != 'g' and node.value != 'G' \
    and node.value != 's' and node.value != 'S':
    step_cost = node.value

  return parent.pathcost + step_cost

def scaledManhattanMinusOne(node, smallestStep, goalList):   #heuristic function to calculate (manhattan distance - 1) between 2 nodes
  goalHeuristics = []                          
  for goal in goalList:
    #Calculates the (manhattan distance - 1) with set minimum of 0 so the heuristic cannot be negative
    #heuristic is scaled to increment by the smallest step increment 
    goalHeuristics.append(smallestStep * max(0, (abs(node.coords[0] - goal.coords[0]) + abs(node.coords[1] - goal.coords[1]) - 1)))
  return min(goalHeuristics)    #returns minimum (manhattan distance -1) to any goal node

def format_grid(filePath, nodeArray, goalArray): #gets input text and formats it

  #Calculates the adjacent nodes for each spot in the grid and store in array
  #Output: array of arrays of nodes

  with open(filePath, encoding="utf8") as fp: #Reads the input CSV and creates lists of nodes for traversal 
    row = 0
    smallestStep = -1
    line = fp.readline()
   
    while line:
      nodeArray.append([])
      processed = line.strip().split(',')   #creates a node for every character in CSV
      for col in range(0, len(processed)):      #format grid and initialize nodes
        currnode = node((row, col), processed[col].strip())
        currnode.pathcost = -1
        nodeArray[row].append(currnode)
        if currnode.value == 's' or currnode.value == 'S':
          currnode.pathcost = 0
          startNode = currnode                    #initializing start node
        elif currnode.value == 'g' or currnode.value == 'G':
          goalArray.append(currnode)             #adding goal node to list of all goals
        elif currnode.value != 'x' and currnode.value != 'X':
          currnode.value = float(currnode.value)
          if smallestStep == -1 or currnode.value < smallestStep:
            smallestStep = currnode.value

        if col > 0:                         #Setting up adjacency lists of nodes to traverse grid 
          leftnode = nodeArray[row][col-1]
          leftnode.addAdjacent(currnode)
          currnode.addAdjacent(leftnode)
        if row > 0:
          uppernode = nodeArray[row-1][col]
          uppernode.addAdjacent(currnode)
          currnode.addAdjacent(uppernode)

      row += 1
      line = fp.readline()

  return (startNode, smallestStep)


#pathfinding("/input.txt", "/optimal_path.txt", "/explored_list.txt")   ---> Test case to run program